In [363]:
FORM_MAPS = {
    "le génitif": "genitive",
    "gen.": "genitive",
    "gén.": "genitive",
    "gén": "genitive",
    "dat.": "dative",
    "dat": "dative",
    "plur.": "plural",
    "plur": "plural",
    "futur": "future",
    "fut.": "future",
    "participe": "participle",
    "comp.": "comparative",
    "comp": "comparative",
    "impér.": "imperative",
    "impér": "imperative"
}

In [429]:
import json
with open("sjoestedt_phonetique.json") as inf:
    DATA = json.load(inf)

In [424]:
section = 30

if not section in DATA:
    DATA[section] = []

In [384]:
DATA[section] = []

In [364]:
import re
REGEX_CASE = rf"^([^(]+) \(([^)]+)\) « ([^»]+) », ({'|'.join(FORM_MAPS.keys())}) ([^(]+) \(([^)]+)\)$"
REGEX_CASE_OF = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) « ([^»]+) »$"
REGEX_CASE_OF_BOTH = rf"^([^(]+) \(([^)]+)\),? ({'|'.join(FORM_MAPS.keys())}) de ([^«]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_BASE = r"^([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_JUST_FRENCH = r"^([^(]+) « ([^»]+) »$"
REGEX_FROM = r"^([^(]+) \(([^)]+)\) « ([^»]+) »,? de ([^(]+) \(([^)]+)\) « ([^»]+) »$"
REGEX_IN = r"^([^(]+) \(([^)]+)\), dans ([^«]+) « ([^»]+) »$"
REGEX_OR = r"^([^(]+) \(([^)]+)\) ou ([^(]+) \(([^)]+)\) « ([^»]+) »$"

def extract(text, splitter, counter):
    text = text.replace("\u00ad", "").replace("\n", " ")
    for p in text.split(splitter):
            m = re.match(REGEX_BASE, p.strip())
            mjf = re.match(REGEX_JUST_FRENCH, p.strip())
            mf = re.match(REGEX_FROM, p.strip())
            min = re.match(REGEX_IN, p.strip())
            mc = re.match(REGEX_CASE, p.strip())
            mcofb = re.match(REGEX_CASE_OF_BOTH, p.strip())
            mcof = re.match(REGEX_CASE_OF, p.strip())
            mor = re.match(REGEX_OR, p.strip())
            if m:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": m.group(1),
                    "irish": m.group(2),
                    "french": m.group(3),
                })
                counter += 1
            elif mjf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mjf.group(1),
                    "french": mjf.group(2),
                })
                counter += 1
            elif mf:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(1),
                    "irish": mf.group(2),
                    "french": mf.group(3).replace("\xad", ""),
                    "from": mf.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mf.group(4),
                    "irish": mf.group(5),
                    "french": mf.group(6).replace("\xad", ""),
                })
                counter += 1
            elif mor:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(1),
                    "irish": mor.group(2),
                    "french": mor.group(5),
                    "alt": mor.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mor.group(3),
                    "irish": mor.group(4),
                    "french": mor.group(5),
                    "alt": mor.group(2)
                })
                counter += 1
            elif min:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(1),
                    "irish": min.group(2),
                    "in": min.group(3)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": min.group(3),
                    "french": min.group(4),
                })
                counter += 1
            elif mc:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(1),
                    "irish": mc.group(2),
                    "french": mc.group(3),
                    FORM_MAPS[mc.group(4)]: mc.group(6)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mc.group(5),
                    "irish": mc.group(6),
                    f"{FORM_MAPS[mc.group(4)]}_of": mc.group(2)
                })
                counter += 1
            elif mcofb:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcofb.group(1),
                    "irish": mcofb.group(2),
                    f"{FORM_MAPS[mcofb.group(3)]}_of": mcofb.group(5)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "irish": mcofb.group(5),
                    "transcription": mcofb.group(4),
                    "french": mcofb.group(6),
                    FORM_MAPS[mcofb.group(3)]: mcofb.group(2)
                })
                counter += 1
            elif mcof:
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(1),
                    "irish": mcof.group(2),
                    f"{FORM_MAPS[mcof.group(3)]}_of": mcof.group(4)
                })
                counter += 1
                DATA[section].append({
                    "section": section,
                    "id": f"{section}_{counter}",
                    "transcription": mcof.group(4),
                    "french": mcof.group(5),
                    FORM_MAPS[mcof.group(3)]: mcof.group(2)
                })
                counter += 1
            else:
                print(p)
    print(counter)


In [615]:
REST = {
    57: "ɑ꞉ⁱfʹe꞉ʃ (áiféis) « exagé­ration » ; dʹᴇfʹɩrʹ (deithbhir) « hâte » ; ɛfʹɩgʹ (oífig) « bureau » ; stɪfʹ (stuif) « substance, matériel »",
}

In [616]:
# DATA[section] = []
section = 57
if not section in DATA:
    DATA[section] = []
extract(REST[section], ";", 12)

16


In [614]:
DATA[section][-1]["transcription"] = DATA[section][-1]["transcription"].split(" ou ")
# DATA[section][-1]["french"] = [DATA[section][-1]["french"], "grimace"]
# DATA[section][-1]["transcription"] = ["ə vɑ꞉hɩrʹ", "ə ṽɑ̃꞉hɩrʹ", "ə vɑ̃꞉hɩrʹ"]

In [617]:
import json

if str(section) in DATA:
    tmp = DATA[str(section)] + DATA[section]
    DATA[str(section)] = tmp
    del(DATA[section])

with open("sjoestedt_phonetique.json", "w") as outf:
    json.dump(DATA, outf)

In [538]:
# DATA[section] = DATA[section][0:5]
del(DATA[section][-1])

In [540]:
# DATA[section][-2]["dative"] = "muic"
# DATA[section][-2]["genitive"] = "luinge"
# del(DATA[section][-1]["dative"])
DATA[section][-2]

{'section': 47,
 'id': '47_9',
 'irish': 'long',
 'transcription': 'lɔŋg',
 'french': 'navire',
 'dative': 'luing',
 'genitive': 'luinge'}